In [3]:
import numpy as np
import cv2
from skimage.transform import resize
import matplotlib.pyplot as plt

In [1]:
def classify(image):
    z1_val = np.dot(image, W1) + b1
    #ass=z1_val-np.dot(X_val, W1)
    a1_val = sigmoid(z1_val)
    z2_val = np.dot(a1_val, W2) + b2
    a2_val = sigmoid(z2_val)
    z3_val = np.dot(a2_val, W3) + b3
    a3_val = sigmoid(z3_val)
    z4_val = np.dot(a3_val, W4) + b4

    y_val_pred = sigmoid(z4_val)
    return np.argmax(y_val_pred,axis=1)

In [2]:
def prob(image):
    z1_val = np.dot(image, W1) + b1
    #ass=z1_val-np.dot(X_val, W1)
    a1_val = sigmoid(z1_val)
    z2_val = np.dot(a1_val, W2) + b2
    a2_val = sigmoid(z2_val)
    z3_val = np.dot(a2_val, W3) + b3
    a3_val = sigmoid(z3_val)
    z4_val = np.dot(a3_val, W4) + b4

    y_val_pred = sigmoid(z4_val)
    return np.array(np.exp(y_val_pred)/np.sum(np.exp(y_val_pred))*100).reshape(26)

In [4]:
W1 = np.load('W1.npy')
W2 = np.load('W2.npy')
W3 = np.load('W3.npy')
W4 = np.load('W4.npy')
b1 = np.load('b1.npy')
b2 = np.load('b2.npy')
b3 = np.load('b3.npy')
b4 = np.load('b4.npy')

In [5]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [6]:
def contt(fgin):
    letters = []
    cdetails = []

    contours, _ = cv2.findContours(np.uint8(fgin), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        cdetails.append(cv2.boundingRect(contour))
        if w > 0 and h > 0:  # Filter out small noise
            letter = np.uint8(fg)[y:y+h, x:x+w]
            # Resize letter while maintaining aspect ratio
            new_size = 28  # Set the size to 28x28 pixels
            aspect_ratio = float(w) / h
            if aspect_ratio > 1.0:
                new_w = new_size
                new_h = int(new_size / aspect_ratio)
            else:
                new_w = int(new_size * aspect_ratio)
                new_h = new_size
            letter = cv2.resize(letter, (new_w, new_h))

            # Create a blank 28x28 canvas and paste the letter in the center
            canvas = 255 * np.zeros((28, 28), dtype=np.uint8)
            x_offset = (28 - new_w) // 2
            y_offset = (28 - new_h) // 2
            canvas[y_offset:y_offset+new_h, x_offset:x_offset+new_w] = letter

            # Save or process the individual letter as needed
            letters.append(canvas)

    border_size = 7
    for ii in range(len(letters)):
        image=letters[ii]
        # Calculate the new dimensions
        new_height = image.shape[0] + 2 * border_size
        new_width = image.shape[1] + 2 * border_size

        # Create a new NumPy array with the specified dimensions, filled with zeros
        new_image = np.zeros((new_height, new_width))

        # Paste the original image into the center of the new array
        new_image[border_size:border_size + image.shape[0], border_size:border_size + image.shape[1]] = image

        image1 = new_image  # Replace with your data

        # Define the desired size of the final image (28x28 in this example)
        final_height = 28
        final_width = 28

        # Use the resize function to resize the image while preserving content
        letters[ii] = resize(image1, (final_height, final_width), anti_aliasing=True)
        
    return letters,cdetails

In [7]:
#UI 2 (Live Updates)

import pygame
import sys

pygame.init()

WIDTH, HEIGHT = 784, 784
GRID_SIZE = 150
CELL_SIZE = WIDTH // GRID_SIZE
pixels = np.zeros((WIDTH, HEIGHT, 3), dtype=np.uint8)
saver = pixels
grid = np.zeros((GRID_SIZE, GRID_SIZE))
draww = False
mpc = [0,0]
reso = GRID_SIZE 

enn=(WIDTH/reso)*np.arange(reso)

spress=0

font = pygame.font.Font(None, 50)
font2 = pygame.font.Font(None, 20)

screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("MNIST Canvas")

tenthframe = 5

def i2c(n):
    return chr(ord('A') + n)

def pen(para1,para2):
    mp = np.array(pygame.mouse.get_pos())
    mpc=(mp*reso)//WIDTH+[para1,para2]
    if(np.min(mpc)>=0):
        pixels[int(np.floor(mpc[0]/GRID_SIZE*WIDTH)):int(np.ceil((mpc[0]+1)/GRID_SIZE*WIDTH)),int(mpc[1]/GRID_SIZE*WIDTH):int((mpc[1]+1)/GRID_SIZE*WIDTH)]=(255,255,255)
    
while (True):
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        elif event.type == pygame.MOUSEBUTTONDOWN:
            draww=True
        elif event.type == pygame.MOUSEBUTTONUP:
            draww=False
        elif event.type == pygame.KEYDOWN and event.key == pygame.K_c:
            pixels = np.zeros((WIDTH, HEIGHT, 3), dtype=np.uint8)
            spress = 0
            dimlist = []
            letterslist = []
        elif event.type == pygame.KEYDOWN and event.key == pygame.K_s:
            saver = pixels
            saver2 = fg
#             print(classify(fg.reshape(784)))
            letterslist, dimlist = contt(fg)
            a = 1
            spress = 1
            
    if(draww):
        pen(0,0)
        pen(1,1)
        pen(-1,-1)
        pen(2,2)
        pen(0,-1)
        pen(1,0)
        pen(-1,-2)
        pen(2,1)
        pen(0,1)
        pen(1,2)
        pen(-1,0)
        pen(2,3)
        
    imggg=np.rot90(np.rot90(np.rot90(pixels))) 
    a=0
    b=0
    grids=np.zeros((reso,reso))
    for i in enn:
        for j in enn:
            grids[a,b]=imggg[int(i),int(j),0]
            a+=1
        b+=1
        a=0
    fg=np.rot90(np.rot90(np.rot90(grids)))/255
        
    
    screen.fill((0,0,0))
    pygame.surfarray.blit_array(screen, pixels)
    
    if (tenthframe==0):
        saver = pixels
        saver2 = fg
        letterslist, dimlist = contt(fg)
        a = 1
        spress = 1
        
    tenthframe=(tenthframe+1)%10
        
    if spress==1 :
        for letterrec in range(len(dimlist)) : 
            rect_x, rect_y, rect_width, rect_height = dimlist[letterrec]
            rect_x, rect_y, rect_width, rect_height = tuple(x * WIDTH // reso for x in (rect_x, rect_y, rect_width, rect_height))
            pygame.draw.rect(screen, (255, 0, 0), (rect_x, rect_y, rect_width, rect_height), 3)  
            
            guessed = classify(letterslist[letterrec].reshape(784))
            text = f'{i2c(int(guessed))}'
            text_surface = font.render(text, True, (255,255,112))
            text_rect = text_surface.get_rect(center=(rect_x, rect_y))
            screen.blit(text_surface, text_rect)
    
    pygame.display.flip()
    pygame.display.update()

pygame 2.5.2 (SDL 2.28.3, Python 3.11.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


C:\Users\pranj\AppData\Local\Temp\ipykernel_11676\4141966738.py:107: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  text = f'{i2c(int(guessed))}'


SystemExit: 

C:\Users\pranj\AppData\Local\Programs\Python\Python311\Lib\site-packages\IPython\core\interactiveshell.py:3450: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
